# 一、BeautifulSoup 爬虫

   * requests 用来获取页面内容
   * BeautifulSoup

* 安装           
pip install requests         
pip install bs4

## 1.爬取链家租房的信息

In [3]:
import requests
from bs4 import BeautifulSoup

In [58]:
#获取url页面内容
def get_page(url,headers):
    r = requests.get(url, headers = headers)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text)
    return soup

#获取整页的每个租房信息的链接
def get_links(link_url,headers):
    soup = get_page(link_url,headers)
    links_a = soup.find_all('a' , attrs = {'class':'content__list--item--aside'})
    links = [('https://cd.lianjia.com'+ link.get('href')) for link in links_a]
    return links

In [49]:
#获取房屋的数据
def get_house_info(house_url):
    headers['Referer']=house_url
    soup = get_page(house_url,headers)
    try:
        name = soup.find('p', class_ = 'content__title').text
        price = soup.find('p', class_ = 'content__aside--title').text.strip()
        #text与.string的区别 https://zhuanlan.zhihu.com/p/30911642
        #个人建议基本用.text不容易出错

        #unit = soup.find('span', class_ = 'unit').text.strip()  #价格单位并去除首尾格式
        house_tag0 = soup.find('p', class_ = 'content__aside--tags').text.strip().split('\n')
        house_tag = ''
        for tag in house_tag0 :
            house_tag = house_tag + tag + ' '

        house_info = soup.find('p', class_ = 'content__article__table').text.strip().split('\n')
        way = house_info[0]
        layout = house_info[1]
        area = house_info[2]
        direction = house_info[3]

        #楼层信息在房屋信息栏里，需要切片访问
        floor = soup.find('div',  class_  =  'content__article__info').text.strip().split('\n')[9][3:]
    except:
        return {}
    info = {
        '标题':name,
        '价格': price,
        '标签':house_tag,        
        '租法':way,
        '户型':layout,
        '面积':area,        
        '朝向':direction, 
        '楼层':floor,
    }
    return info


In [55]:
#抓取的文件写入本地
def outToFile(dict):
    '''
    写入本地当前目录的cdlianjia.txt文件中
    '''
    if dict:
        with open('cdlianjia.txt','a+',encoding='utf-8') as f:
            f.write('标题:{}\t 价格:{}\t 标签:{}\t 租法:{}\t 户型:{}\t 面积:{}\t 朝向:{}\t 楼层:{}\n'.
                    format(dict['标题'],dict['价格'], dict['标签'], dict['租法'], dict['户型'], 
                           dict['面积'], dict['朝向'], dict['楼层']))
            


In [59]:
def start(deep):
    #构造headers模拟真实浏览器访问
    headers = {
        'Host': "cd.lianjia.com",
        'Accept': "application/json, text/javascript, */*; q=0.01",
        'Accept-Encoding': "gzip, deflate, br",
        'Accept-Language': "zh-CN,zh;q=0.9",
        'Referer': 'https://cd.lianjia.com/zufang/',
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",
        'Connection': "keep-alive",
    }
    url_list = [('https://cd.lianjia.com/zufang/'+'pg'+str(k) +'/') for k in range(1,deep+1)]
    print('已获取链接，开始爬虫')
    for url in url_list:
        headers['Referer'] = url
        links = get_links(url, headers)  #得到每个房屋的链接        
        for link in links :
            house_info = get_house_info(link)  #得到单个租房信息字典
            outToFile(house_info)
        print('当前网页爬取完毕')
    print('所有页已爬取完毕')
        

In [60]:
start(4)

已获取链接，开始爬虫
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
所有页已爬取完毕


错误总结：    
1. get('href')写成了get('herf')，于是截取不到链接    
2. 'NoneType' object has no attribute 'text'错误，是我们提取的时候有空的信息，要进行判断处理

## 2.爬取百度贴吧逆水寒吧的帖子信息

In [62]:
import requests
import time
from bs4 import BeautifulSoup

In [63]:
#获取网页

def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        # 这里我们知道百度贴吧的编码是utf-8，所以手动设置的。爬去其他的页面时建议使用如下代码，但是该编码方式不一定准确：
        #r.encoding = r.apparent_encoding
        r.encoding = 'utf-8'
        return r.text
        #html=r.content
        #html_doc=str(html,'utf-8')         #html_doc=html.decode("utf-8","ignore")
        #return html_doc
    except:
        return " ERROR "

这里我们知道 r.text 返回的是Unicode型的数据。    
使用r.content返回的是bytes型的数据。   
也就是说，如果你想取文本，可以通过r.text。  
如果想取图片，文件，则可以通过r.content。   

<pre/>
    r = requests.get(url, timeout = 30)
    r.raise_for_status()
    html=r.content
    html_doc=str(html,'utf-8')         #html_doc=html.decode("utf-8","ignore")
    return html_doc
</pre>

In [64]:
#获取信息
def get_content(url):
    
    #初始化一个列表用来保存所有帖子信息
    posts = []
    #下载网页到本地
    html_doc = get_html(url)

    
    
    #a = requests.get(url)
    #html=a.content
    #html_doc=str(html,'utf-8') 
    #html_doc=html.decode("utf-8","ignore")
    #做一锅汤
    soup = BeautifulSoup(html_doc)

    #找到所有具有 'j_thread_list clearfix'属性的li标签，存入列表
    liTags = soup.find_all('li', attrs={'class':'j_thread_list'} )

    #寻找信息

    for li in liTags:
        #字典存储文章信息
        post = {}

        #使用try 防止爬虫找不到信息从而停止运行
        try :
            '''
            post['title'] = li.find('a', attrs = {'class':'threadlist_title pull_left j_th_tit '}).text.strip()
            post['link'] = "http://tieba.baidu.com/"+li.find('a', attrs={'class': 'threadlist_title pull_left j_th_tit '})['href']
            post['name'] = li.find('span', attrs={'class': 'tb_icon_author '}).text.strip()
            post['time'] = li.find('span', attrs = {'class':'threadlist_reply_date pull_right j_reply_data'}).text.strip()
            post['replyNum'] = li.find('span', sttrs = {'class':'threadlist_reply_date pull_right j_reply_data'}).text.strip()
            '''
            post['title'] = li.find('a', class_='j_th_tit').text
            post['link'] = 'http://tieba.baidu.com' + li.find('a', class_='j_th_tit').get('href')
            post['name'] = li.find('a', class_= 'j_user_card').text
            post['time'] = li.find('span', class_='is_show_create_time').text
            post['replyNum'] = li.find('span', class_='threadlist_rep_num').text
            posts.append(post)
        except:
            print('出问题了~')
    return posts

In [65]:
#抓取的文件写入本地
def outToFile(dict):
    '''
    写入本地当前目录的nshba.txt文件中
    '''
    with open('nshba.txt','a+',encoding='utf-8') as f:
        for post in dict:
            f.write('标题:{}\t 链接:{}\t 发帖人:{}\t 最后回复时间:{}\t 回复数量:{}\n'.format(
                post['title'], post['link'], post['name'], post['time'], post['replyNum']))
            
        print('当前网页爬取完毕')

In [66]:
def start(url_0, deep):
    
    url_list = [] #将要爬取的url存入列表

    for i in range(0,deep):
        url_list.append(url_0 +'&pn=' + str(50 * i))
    print('所有网页已经下载到本地了，开始爬虫吧')
    
    #循环写入信息
    for url in url_list:
        post = get_content(url)
        outToFile(post)
    print('所有信息都爬取并保存完毕')

In [67]:
url_0 = 'https://tieba.baidu.com/f?kw=%E9%80%86%E6%B0%B4%E5%AF%92ol&ie=utf-8'
deep = 3
start(url_0, deep)

所有网页已经下载到本地了，开始爬虫吧
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
所有信息都爬取并保存完毕
